In [2]:
import pandas as pd
import numpy as np
import re
import sys, os
cdir = os.getcwd()
sys.path.insert(0, cdir+'/scraping_tools')
#from browserinstance import BrowserInstance
#import dask.dataframe as dd

In [3]:
urls = pd.read_csv("candidate_url.txt",names=["URL"]).reset_index()
urls = urls.rename(columns={'index':'jobNum'})
urls['jobNum']=urls['jobNum']+1

Creates m2, a dataframe of the names, urls, text, and hashes

In [4]:
dirpath = '/scratch/group/oit_research_data/candidates_2020/'

run1 = 'output_files_15067583/'
run2 = 'output_files_15070517/'

run1urls = pd.read_csv(dirpath+run1+'test.csv',names=['jobNum','URL','hash'])
run2urls = pd.read_csv(dirpath+run2+'test.csv',names=['jobNum','URL','hash'])

m1 = pd.merge(urls,run1urls, on=['jobNum','URL'], how="left")
m2 = pd.merge(m1,run2urls, on=['jobNum','URL'], how="left")

m2['hash_x'] = m2['hash_x'].fillna(m2['hash_y'])

m2 = m2.drop(columns=['hash_y'])
m2 = m2.rename(columns={'hash_x':'hash'})

In [5]:
m2

,jobNum,URL,hash
0,1,https://jamesaverhart.com/meet-james/,9816d88b76bca333830f3243a2b9095cd2a45a7fc40b8b...
1,2,https://harveyhall2congress.com/meet-phyllis/#...,39b6b4efe415004e0de4d3690238ea420ba8b8343c9fa6...
2,3,https://www.winfrey4congress.com/meet-dr-dia,92bb52344b172ef964746c21b0b5554d8a07e68141423a...
3,4,https://neighbors4congress.com/meet-rick-neigh...,358662f3901ac0d0bbd82c192b3134621775c1e1f205ba...
4,5,https://sewellforcongress.com/about-terri/,1e87955c9e1232f69e3ffb474d8fe501538a0a186f450b...
5,6,https://www.tomohalleran.com/about-tom,9bc4ae3eaa4180677c376d97cda12af4befc49f996cbec...
6,7,https://www.kirkpatrickforcongress.com/about/,c997bd0dd19884b9d995425a1168628ed2f0142925be93...
7,8,https://standwithraul.com/bio/,2632f2ee383645eb8247d7844bfe2732aec1f010c93837...
8,9,https://www.delinadisanto.com/about,cd5294fb2615402a10a91faf5e9bc557a6f134f30131a5...
9,10,https://www.joangreeneaz.com/about,d7d440bc0656ca4a39d4957eb2ef095785b140ab46cc2a...


In [4]:
archive = '/scratch/group/oit_research_data/candidates_2020/data_hash/'
browser = BrowserInstance(save_directory=archive, tor=False)

In [5]:
i = 79
print(m2.loc[i])

h = m2['hash'][i]+'.txz'
#print(h)

s = browser.get_page_from_archive(archive+h)
string = " ".join(re.split('\t|\n',str(s)))
stringlist = re.split(r'<.*?\>',string)
[el.strip() for el in stringlist if len(el.strip())>5 and "{" not in el and "[" not in el and "_" not in el and "=" not in el and "||" not in el and "&&" not in el]

jobNum                                                   80
URL                https://dianeforcolorado.com/meet-diane/
hash      92ae73309164b1f7473a79fa3c52e5e6ea82c9c6d397ca...
Name: 79, dtype: object


TimeoutException: Message: 


In [16]:
print(m2.loc[78][2])

5ae6ca59833ee7861fbda11b06c0c8fccf54c402fb3f16421073a7592e510d6b


In [ ]:
def get_page_text(i,archive=archive):
    
    if pd.isnull(m2['hash'][i]):
        return "None"
    else:
        h = m2['hash'][i]+'.txz'
    
    try:
        s = browser.get_page_from_archive(archive+h)
    except:
        return "[Filler Text]"
    string = " ".join(re.split('\t|\n',str(s)))
    stringlist = re.split(r'<.*?\>',string)
    text = [el.strip() for el in stringlist if len(el.strip())>5 and "{" not in el and "[" not in el and "_" not in el and "=" not in el and "||" not in el and "&&" not in el]
    
    return text

In [ ]:
m2['PageText']=m2.apply(lambda x: get_page_text(x.name), axis=1)

In [ ]:
m2[['jobNum', 'URL','PageText']]

In [ ]:
m2.to_csv("~/candidates_2020/m2.csv", index=False)

Attempt to fix rows with no pagetext

In [7]:
m3=m2

In [8]:
m3['PageText']=m3[80:90].apply(lambda x: get_page_text(x.name), axis=1)

NameError: ("name 'get_page_text' is not defined", 'occurred at index 80')

In [9]:
m3[80:90]

,jobNum,URL,hash
80,81,https://ike4co.com/ikebio/,308900306352797e87e10e6de81f70354fb8b40368aa3a...
81,82,https://jillianfreeland.com/about/,cef6a84cdce3a428e647e6a94abc1ebefcb0ef8e042b7c...
82,83,https://jasoncrowforcongress.com/about/,4286a04dc428e2353b0d7ef98781d6017b971beb33cae4...
83,84,https://perlmutter.house.gov/about/biography.htm,6dd4aa1f983d35e2469d9f5dcbc9e0af571ada99fee7cb...
84,85,https://larson.house.gov/about/meet-john,b548f93752d837d6b161f72d2f695025ea545561730f04...
85,86,https://courtney.house.gov/about/full-biography,b028d4dac01a21e3d1208821a41770c44ab10f75b9fd09...
86,87,https://rosadelauro.com/about/,f8c789a1d683cfde7f11c7f837965da5aa0ebe9a06688b...
87,88,https://www.himesforcongress.com/about/,c8dc52219dc5d5830163e9917c38fe3a85adbebf641a57...
88,89,https://jahanahayes.com/about/,6aba065e691e5be7bc2bb18a75557d15a49b8b1a3811af...
89,90,https://lisabluntrochester.com/about,4e3c8e027c952179a31a5f6ec37830dd5876bd47fde787...


Errors are 79,